### Model Development Notebook

Model development would ideally be performed in the Snowflake DW. However, this notebook presents a quick alternative for that purpose.

In [57]:
import duckdb
import pandas as pd
from pathlib import Path
import ipywidgets as widgets
from IPython.display import display, Markdown

# Allow full column display
pd.set_option('display.max_columns', None)

# Helper to run DuckDB query
def run_query(sql_query: str) -> pd.DataFrame:
    db_path = Path('../data_lake/dbt.duckdb')
    try:
        with duckdb.connect(database=str(db_path), read_only=True) as con:
            return con.execute(sql_query).df()
    except Exception as e:
        print(f"An error occurred: {e}")
        return pd.DataFrame()

print("Helper function `run_query` is defined.")

# Input and button
model_input = widgets.Text(
    value='dim_calendar',
    description='Model:',
    style={'description_width': '80px'},
    layout=widgets.Layout(width='300px', margin='0 10px 0 0')
)

run_button = widgets.Button(
    description='Load Model',
    button_style='primary',
    layout=widgets.Layout(width='120px', height='35px')
)

# Outputs
model_output = widgets.Output(layout=widgets.Layout(width='700px'))  # fixed width
tables_output = widgets.Output(layout=widgets.Layout(width='320px'))  # fixed width

# Preload table list
with tables_output:
    display(Markdown("### Available Tables/Models in `main` schema"))
    display(run_query("SHOW TABLES"))

# Button callback
def on_run_clicked(b):
    with model_output:
        model_output.clear_output()
        model_name = model_input.value

        display(Markdown(f"<h3 style='color:#5C2D91;'>Loading Model: <code style='font-size: 0.8em; background-color:#eee;'>{model_name}</code></h3>"))

        transactions_df = run_query(f"SELECT * FROM main.{model_name}")

        display(Markdown("#### Schema (Data Types)"))
        schema_df = pd.DataFrame({
            'Column': transactions_df.columns,
            'Non-Null Count': transactions_df.notnull().sum().values,
            'Dtype': transactions_df.dtypes.values
        })
        display(schema_df)

        display(Markdown("#### First 5 Rows"))
        display(transactions_df)

run_button.on_click(on_run_clicked)

# UI layout
input_row = widgets.HBox([model_input, run_button])

content_row = widgets.HBox(
    [model_output, tables_output],
    layout=widgets.Layout(
        justify_content='flex-start',
        align_items='flex-start',
        gap='10px',  # small gap
        width='100%'
    )
)

# Render full UI
display(widgets.VBox([input_row, content_row]))


Helper function `run_query` is defined.


In [24]:
sql_input = widgets.Textarea(
    value="""SELECT * FROM main.dim_calendar LIMIT 10""",
    placeholder='Enter your SQL query here...',
    description='SQL:',
    layout=widgets.Layout(width='1000px', height='auto'),
    style={'description_width': '40px'}
)

# Button to run SQL
run_sql_button = widgets.Button(
    description='Run SQL',
    button_style='success',
    layout=widgets.Layout(width='120px', height='auto')
)

# Output area
sql_output = widgets.Output()

# Callback function
def on_sql_run_clicked(b):
    with sql_output:
        sql_output.clear_output()
        try:
            df = run_query(sql_input.value)
            display(Markdown("### Query Results"))
            display(df)
        except Exception as e:
            print(f"Error: {e}")

run_sql_button.on_click(on_sql_run_clicked)

# Display the full SQL UI
display(widgets.VBox([sql_input, run_sql_button, sql_output]))


In [51]:
sql = """
WITH daily_aggregates AS (
    -- First, aggregate the raw transactions to a daily level.
    SELECT
        request_id,
        email,
        date,
        -- Use the date calculation columns from the source table
        most_recent_statement_date,
        most_recent_statement_date_minus_90_days,
        most_recent_statement_date_minus_180_days,
        SUM(IF(is_revenue, deposits, 0)) as daily_revenue,
        SUM(IF(is_debit, withdrawals, 0)) as daily_debits
    FROM main.all_transactions_by_customer
    GROUP BY ALL
)

-- ,daily_balances AS (
    SELECT *
--        request_id,
--        email,
--        ROUND(AVG(revised_average_balance), 2) AS average_daily_balance
    FROM main.fct_daily_transactions_by_customer
--    WHERE date > most_recent_statement_date_minus_180_days
    WHERE date >= '2024-01-05' AND date <= '2024-01-13'
    ORDER BY date DESC

--    GROUP BY ALL
--)

-- ,most_recent_balances AS (
--     SELECT
--         request_id,
--         email,
--         balance AS most_recent_balance
--     FROM main. all_transactions_by_customer
--     QUALIFY ROW_NUMBER() OVER(PARTITION BY request_id, email ORDER BY date DESC) = 1
-- )

-- SELECT
--     dag.request_id,
--     dag.email,

--     -- Revenue Metrics
--     SUM(dag.daily_revenue) AS revenue_total_credit,
--     SUM(dag.daily_revenue) AS revenue_total,
--     SUM(IF(dag.date >= dag.most_recent_statement_date_minus_90_days, dag.daily_revenue, 0)) AS revenue_recent_90_days,
--     SUM(IF(dag.date >= dag.most_recent_statement_date_minus_180_days AND dag.date < dag.most_recent_statement_date_minus_90_days, dag.daily_revenue, 0)) AS revenue_91_to_180_days,

--     -- Debit Metrics
--     SUM(dag.daily_debits) AS debits_total,
--     SUM(IF(dag.date >= dag.most_recent_statement_date_minus_90_days, dag.daily_debits, 0)) AS debits_recent_90_days,
--     SUM(IF(dag.date >= dag.most_recent_statement_date_minus_180_days AND dag.date < dag.most_recent_statement_date_minus_90_days, dag.daily_debits, 0)) AS debits_91_to_180_days,

--     -- Placeholder Credit Metrics
--     0 AS credit_card_payments,
--     0 AS credit_card_recent_90_days,
--     0 AS credit_card_91_to_180_days,

--     -- Averages and Balances
--     db.average_daily_balance AS average_daily_balance_across_bank_accounts,
--     mrb.most_recent_balance AS most_recent_balance_across_bank_accounts
-- FROM daily_aggregates AS dag
-- LEFT JOIN daily_balances AS db USING(request_id, email)
-- LEFT JOIN most_recent_balances AS mrb USING(request_id, email)
-- GROUP BY ALL

"""
df = run_query(sql)
df.head(12)



,email,request_id,date,revised_average_balance,daily_revenue,weekly_revenue,most_recent_statement_date,most_recent_statement_date_minus_30_days,most_recent_statement_date_minus_60_days,most_recent_statement_date_minus_90_days,most_recent_statement_date_minus_180_days,most_recent_statement_date_minus_365_days
0,JOELSCHAUBEL@GMAIL.COM,727DAE61-63E9-4121-801E-F11CA8FF32FD,2024-01-13,2265.55,NaN,NaN,2024-02-09,2024-01-10,2023-12-11,2023-11-11,2023-08-13,2023-02-09
1,JOELSCHAUBEL@GMAIL.COM,727DAE61-63E9-4121-801E-F11CA8FF32FD,2024-01-12,2265.55,NaN,NaN,2024-02-09,2024-01-10,2023-12-11,2023-11-11,2023-08-13,2023-02-09
2,JOELSCHAUBEL@GMAIL.COM,727DAE61-63E9-4121-801E-F11CA8FF32FD,2024-01-11,2185.98,437.0,3723.0,2024-02-09,2024-01-10,2023-12-11,2023-11-11,2023-08-13,2023-02-09
3,JOELSCHAUBEL@GMAIL.COM,727DAE61-63E9-4121-801E-F11CA8FF32FD,2024-01-10,2097.13,NaN,NaN,2024-02-09,2024-01-10,2023-12-11,2023-11-11,2023-08-13,2023-02-09
4,JOELSCHAUBEL@GMAIL.COM,727DAE61-63E9-4121-801E-F11CA8FF32FD,2024-01-09,2524.44,NaN,NaN,2024-02-09,2024-01-10,2023-12-11,2023-11-11,2023-08-13,2023-02-09
5,JOELSCHAUBEL@GMAIL.COM,727DAE61-63E9-4121-801E-F11CA8FF32FD,2024-01-08,1516.64,3286.0,3723.0,2024-02-09,2024-01-10,2023-12-11,2023-11-11,2023-08-13,2023-02-09
6,JOELSCHAUBEL@GMAIL.COM,727DAE61-63E9-4121-801E-F11CA8FF32FD,2024-01-07,523.41,NaN,NaN,2024-02-09,2024-01-10,2023-12-11,2023-11-11,2023-08-13,2023-02-09
7,JOELSCHAUBEL@GMAIL.COM,727DAE61-63E9-4121-801E-F11CA8FF32FD,2024-01-06,523.41,NaN,NaN,2024-02-09,2024-01-10,2023-12-11,2023-11-11,2023-08-13,2023-02-09
8,JOELSCHAUBEL@GMAIL.COM,727DAE61-63E9-4121-801E-F11CA8FF32FD,2024-01-05,523.41,NaN,NaN,2024-02-09,2024-01-10,2023-12-11,2023-11-11,2023-08-13,2023-02-09


In [56]:
sql = """
-- This model creates a complete, daily time series for each customer over the
-- last 180 days, filling in any missing dates with the last known balance.

WITH all_daily_transactions AS (
    SELECT
        username,
        email,
        request_id,
        request_datetime,
        date,
        withdrawals,
        deposits,
        balance,
        is_revenue,
        is_debit,
        most_recent_statement_date,
        most_recent_statement_date_minus_30_days,
        most_recent_statement_date_minus_60_days,
        most_recent_statement_date_minus_90_days,
        most_recent_statement_date_minus_180_days,
        most_recent_statement_date_minus_365_days
    FROM main.all_transactions_by_customer
)

,dim_calendar AS (
    SELECT *
    FROM main.dim_calendar
)

,customer_date_range AS (
    SELECT
        email,
        request_id,
        date,
        
        -- Use 180 days as default for scaffolding, but this can be easily changed
        most_recent_statement_date_minus_365_days AS start_date,
        most_recent_statement_date AS end_date,
        
        -- Include all auxiliary date columns for reference
        most_recent_statement_date,
        most_recent_statement_date_minus_30_days,
        most_recent_statement_date_minus_60_days,
        most_recent_statement_date_minus_90_days,
        most_recent_statement_date_minus_180_days,
        most_recent_statement_date_minus_365_days
    FROM all_daily_transactions
    GROUP BY ALL
)

,customer_scaffold AS (
    SELECT
        cdr.email,
        cdr.request_id,
        cal.date_day as date
    FROM customer_date_range AS cdr
    CROSS JOIN dim_calendar AS cal
    WHERE cal.date_day > cdr.start_date
        AND cal.date_day <= cdr.end_date
    ORDER By date DESC
)

,padded_transactions AS (
    SELECT
        scf.email,
        scf.request_id,
        scf.date,
        -- Average balance for days with multiple transactions
        AVG(trn.balance) AS average_balance
    FROM customer_scaffold AS scf
    LEFT JOIN all_daily_transactions AS trn ON scf.email = trn.email
        AND scf.request_id = trn.request_id
        AND scf.date = trn.date
    GROUP BY ALL
)

,daily_balances AS (
    SELECT
        email,
        request_id,
        date,
        average_balance,
        
        -- Fill forward the last known balance for days without transactions
        LAST_VALUE(average_balance IGNORE NULLS) OVER(
            PARTITION BY email, request_id
            ORDER BY date
            ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
        ) AS revised_average_balance
    FROM padded_transactions AS trn
)

,daily_revenue AS (
    SELECT
        email,
        request_id,
        date,
        
        -- Sum of all deposits for the day. This is the "Day Rev" from the sheet.
        SUM(deposits) OVER(PARTITION BY email, request_id, DAYOFYEAR(date)) AS daily_revenue,
    FROM all_daily_transactions AS trn
    WHERE is_revenue
    QUALIFY ROW_NUMBER() OVER(PARTITION BY email, request_id, date) = 1
)

,weekly_revenue AS (
    SELECT
        email,
        request_id,
        date,
        
        -- Sum of all deposits for the day. This is the "Weekly revenue" from the sheet.
        SUM(deposits) OVER(PARTITION BY email, request_id, WEEKOFYEAR(date)) AS weekly_revenue,
    FROM all_daily_transactions
    WHERE is_revenue
    QUALIFY ROW_NUMBER() OVER(PARTITION BY email, request_id, date) = 1
)

,customer_daily_metrics AS (
    SELECT
        db.email,
        db.request_id,
        db.date,
        ROUND(db.revised_average_balance, 2) AS revised_average_balance,

        -- Daily and weekly revenues
        drv.daily_revenue,
        wrv.weekly_revenue,
        
        -- Include auxiliary date columns for reference
        ANY_VALUE(cdr.most_recent_statement_date) OVER (PARTITION BY db.email, db.request_id) AS most_recent_statement_date,
        ANY_VALUE(cdr.most_recent_statement_date_minus_30_days) OVER (PARTITION BY db.email, db.request_id) AS most_recent_statement_date_minus_30_days,
        ANY_VALUE(cdr.most_recent_statement_date_minus_60_days) OVER (PARTITION BY db.email, db.request_id) AS most_recent_statement_date_minus_60_days,
        ANY_VALUE(cdr.most_recent_statement_date_minus_90_days) OVER (PARTITION BY db.email, db.request_id) AS most_recent_statement_date_minus_90_days,
        ANY_VALUE(cdr.most_recent_statement_date_minus_180_days) OVER (PARTITION BY db.email, db.request_id) AS most_recent_statement_date_minus_180_days,
        ANY_VALUE(cdr.most_recent_statement_date_minus_365_days) OVER (PARTITION BY db.email, db.request_id) AS most_recent_statement_date_minus_365_days

    FROM daily_balances AS db
    LEFT JOIN daily_revenue AS drv USING(email, request_id, date)
    LEFT JOIN weekly_revenue AS wrv USING(email, request_id, date)
    LEFT JOIN customer_date_range AS cdr USING(email, request_id, date)
)

SELECT *
FROM customer_daily_metrics
WHERE date >= '2024-01-05' AND date <= '2024-01-13'    
ORDER BY date DESC

"""
df = run_query(sql)
df.head(14)

,email,request_id,date,revised_average_balance,daily_revenue,weekly_revenue,most_recent_statement_date,most_recent_statement_date_minus_30_days,most_recent_statement_date_minus_60_days,most_recent_statement_date_minus_90_days,most_recent_statement_date_minus_180_days,most_recent_statement_date_minus_365_days
0,JOELSCHAUBEL@GMAIL.COM,727DAE61-63E9-4121-801E-F11CA8FF32FD,2024-01-13,2265.55,NaN,NaN,2024-02-09,2024-01-10,2023-12-11,2023-11-11,2023-08-13,2023-02-09
1,JOELSCHAUBEL@GMAIL.COM,727DAE61-63E9-4121-801E-F11CA8FF32FD,2024-01-12,2265.55,NaN,NaN,2024-02-09,2024-01-10,2023-12-11,2023-11-11,2023-08-13,2023-02-09
2,JOELSCHAUBEL@GMAIL.COM,727DAE61-63E9-4121-801E-F11CA8FF32FD,2024-01-11,2185.98,437.0,3723.0,2024-02-09,2024-01-10,2023-12-11,2023-11-11,2023-08-13,2023-02-09
3,JOELSCHAUBEL@GMAIL.COM,727DAE61-63E9-4121-801E-F11CA8FF32FD,2024-01-10,2097.13,NaN,NaN,2024-02-09,2024-01-10,2023-12-11,2023-11-11,2023-08-13,2023-02-09
4,JOELSCHAUBEL@GMAIL.COM,727DAE61-63E9-4121-801E-F11CA8FF32FD,2024-01-09,2524.44,NaN,NaN,2024-02-09,2024-01-10,2023-12-11,2023-11-11,2023-08-13,2023-02-09
5,JOELSCHAUBEL@GMAIL.COM,727DAE61-63E9-4121-801E-F11CA8FF32FD,2024-01-08,1516.64,3286.0,3723.0,2024-02-09,2024-01-10,2023-12-11,2023-11-11,2023-08-13,2023-02-09
6,JOELSCHAUBEL@GMAIL.COM,727DAE61-63E9-4121-801E-F11CA8FF32FD,2024-01-07,523.41,NaN,NaN,2024-02-09,2024-01-10,2023-12-11,2023-11-11,2023-08-13,2023-02-09
7,JOELSCHAUBEL@GMAIL.COM,727DAE61-63E9-4121-801E-F11CA8FF32FD,2024-01-06,523.41,NaN,NaN,2024-02-09,2024-01-10,2023-12-11,2023-11-11,2023-08-13,2023-02-09
8,JOELSCHAUBEL@GMAIL.COM,727DAE61-63E9-4121-801E-F11CA8FF32FD,2024-01-05,523.41,NaN,NaN,2024-02-09,2024-01-10,2023-12-11,2023-11-11,2023-08-13,2023-02-09
